## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-13-01-14-48 +0000,nypost,Video shows firefighters working to rescue dog...,https://nypost.com/2026/02/12/us-news/video-sh...
1,2026-02-13-01-12-04 +0000,bbc,Japan says it seized Chinese vessel amid tensi...,https://www.bbc.com/news/articles/c2e4791re38o...
2,2026-02-13-01-07-59 +0000,bbc,Costs from Trump's tariffs paid almost entirel...,https://www.bbc.com/news/articles/c78x9256pn7o...
3,2026-02-13-01-06-37 +0000,nyt,Judge Says Immigrant Detainees Near Minneapoli...,https://www.nytimes.com/2026/02/12/us/minnesot...
4,2026-02-13-01-06-18 +0000,missionlocal,"Tasting menus out, bar food in: Outta Sight Pi...",https://missionlocal.org/2026/02/tasting-menus...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2421/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,42
62,immigration,16
104,epstein,15
51,new,12
78,up,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
95,2026-02-12-22-03-00 +0000,wsj,Border czar Tom Homan said the Trump administr...,https://www.wsj.com/politics/policy/tom-homan-...,140
159,2026-02-12-20-01-49 +0000,nyt,Trump Administration to End Surge of Immigrati...,https://www.nytimes.com/2026/02/12/us/minnesot...,94
162,2026-02-12-19-47-08 +0000,bbc,Minnesota immigration enforcement surge is end...,https://www.bbc.com/news/articles/c2lr9w29zwyo...,93
263,2026-02-12-15-00-00 +0000,wsj,Josh Gruenbaum’s brash ways as a federal agenc...,https://www.wsj.com/politics/national-security...,88
262,2026-02-12-15-03-32 +0000,cbc,Trump border czar says controversial Minneapol...,https://www.cbc.ca/news/world/us-minnesota-hom...,87


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
95,140,2026-02-12-22-03-00 +0000,wsj,Border czar Tom Homan said the Trump administr...,https://www.wsj.com/politics/policy/tom-homan-...
47,73,2026-02-12-23-36-00 +0000,wsj,"Chris Klomp, the head of Medicare and a former...",https://www.wsj.com/politics/policy/white-hous...
265,56,2026-02-12-14-38-32 +0000,nypost,Woman seen in photo under disgraced ex-Prince ...,https://nypost.com/2026/02/12/us-news/woman-in...
341,47,2026-02-12-08-39-36 +0000,wsj,"Stock Market News, Feb. 12, 2026: Tech Shares ...",https://www.wsj.com/livecoverage/stock-market-...
264,47,2026-02-12-14-41-00 +0000,wsj,What a shutdown would mean for the Department ...,https://www.wsj.com/politics/policy/homeland-s...
263,46,2026-02-12-15-00-00 +0000,wsj,Josh Gruenbaum’s brash ways as a federal agenc...,https://www.wsj.com/politics/national-security...
50,39,2026-02-12-23-33-00 +0000,wsj,When Customs and Border Protection fired a las...,https://www.wsj.com/politics/national-security...
205,39,2026-02-12-17-59-02 +0000,nypost,Gail Slater leaving role as DOJ antitrust chie...,https://nypost.com/2026/02/12/business/gail-sl...
46,37,2026-02-12-23-36-00 +0000,wsj,"Air Canada Swings to Profit, Logs Higher Sales...",https://www.wsj.com/business/earnings/air-cana...
319,36,2026-02-12-10-30-00 +0000,wsj,The mixed jobs picture reflects surging demand...,https://www.wsj.com/economy/jobs/what-sweeping...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
